In [6]:
# !pip install -q -U numpy pandas kaggle_environments

In [7]:
import numpy as np
import pandas as pd
import random

from kaggle_environments import make, evaluate
from kaggle_environments.envs.rps.utils import get_score

def printl(text):
    # Обёртка print для агентов
    print(text)

## Опишем поведение нескольких агентов:

In [8]:
# Static agents
def rock_agent(observation, configuration):
    """Агент, который всегда выбирает жест "камень".

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: константа - 0.
    """
    printl("Agent action - 0")
    return 0

def paper_agent(observation, configuration):
    """Агент, который всегда выбирает жест "бумага".

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: константа - 1.
    """
    printl("Agent action - 1")
    return 1

def scissors_agent(observation, configuration):
    """Агент, который всегда выбирает жест "ножницы".

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: константа - 2.
    """
    printl("Agent action - 2")
    return 2

def random_agent(observation, configuration):
    """Агент, который всегда выбирает случайный жест.

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: число из диапазона [0, configuration.signs - 1].
    """
    move = random.randrange(0, configuration.signs)
    printl(f"Agent action {move}")
    return move

# Based on lib
def copy_opponent_agent(observation, configuration):
    """Агент, повторяющий последний жест противника.

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: число из диапазона [0, configuration.signs - 1].
    """
    if observation.step > 0:
        move = observation.lastOpponentAction  
    else:
        move = random.randrange(0, configuration.signs)  
    printl(f"Agent action {move}")
    return move

def respond_to_opponent_agent(observation, configuration):
    """Агент, выбирающий жест, который побеждает предыдущий жест противника.

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: число из диапазона [0, configuration.signs - 1].
    """
    if observation.step == 0:
        move = random.randrange(0, configuration.signs)  
    else:
        move = ( 1 + observation.lastOpponentAction ) % 3
    printl(f"Agent action {move}")  
    return move

agent_sequence_sign=0
agent_sequence_order=[0, 1, 2]

def agent_sequence(observation, configuration):
    """Агент, который выбирающий жесты в строгой последовательности.

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: число из диапазона [0, configuration.signs - 1].
    """
    global agent_sequence_sign
    move = agent_sequence_order[agent_sequence_sign] 
    agent_sequence_sign = (agent_sequence_sign + 1) % len(agent_sequence_order)
    printl(f"Agent action {move}")
    return move

common_selection_agent_stats={0: 0, 1: 0, 2: 0}
def common_selection_agent(observation, configuration):
    """Агент, который выбирающий жесты, противодействующие наиболее частому ходу противника.

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: число из диапазона [0, configuration.signs - 1].
    """
    global common_selection_agent_stats
    if observation.step > 0:
        last_move = observation.lastOpponentAction
        common_selection_agent_stats[last_move] += 1  
    else:
        return random.randrange(0, configuration.signs)  
    most_common_move = max(common_selection_agent_stats, key=common_selection_agent_stats.get)
    move = ( 1 + most_common_move ) % 3
    printl(f"Agent action {move}")
    return move

reward_check_agent_sign = 0
reward_check_agent_order = [2, 2, 2, 0, 1]
reward_check_agent_def = random.sample(reward_check_agent_order, 3)
def reward_check_agent(observation, configuration):
    """Агент, который принимает решения на основе полученной награды и
    последнего хода противника.

    Этот агент использует три различных стратегии в зависимости от награды:
    - `reward <= 25` - жест, противодействующий ходу противника
    - `reward > 25 && reward <= 75` - случайный жест.
    - `reward > 75` - использование заранее определенного порядка

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: число из диапазона [0, configuration.signs - 1].
    """
    global reward_check_agent_sign
    if observation.step == 0:
        move = random.randrange(0, configuration.signs)
    else:
        if observation.reward <= 25:
            opponent_move = observation.lastOpponentAction
            if opponent_move == 0:  
                move = 1 
            elif opponent_move == 1: 
                move = 2 
            elif opponent_move == 2:  
                move = 0     
        elif 25 < observation.reward <= 75:
            move = random.randrange(0, configuration.signs)  
        else:
            move = reward_check_agent_def[reward_check_agent_sign]
            reward_check_agent_sign += 1
            if reward_check_agent_sign >= len(reward_check_agent_def):  
                reward_check_agent_def = random.sample(reward_check_agent_order, 3)  
                reward_check_agent_sign = 0
    printl(f"Agent action {move}")
    return move

anti_replay_agent_last_move = None
def anti_replay_agent(observation, configuration):
    """Агент, который избегает повторения одного и того же хода дважды подряд.

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: число из диапазона [0, configuration.signs - 1].
    """
    global anti_replay_agent_last_move
    if observation.step == 0:
        move = random.choice([0, 1, 2])
    else:
        new_order = [i for i in [0, 1, 2] if i != anti_replay_agent_last_move]
        move = random.choice(new_order)  
    anti_replay_agent_last_move = move
    printl(f"Agent action {move}")
    return move

def predictive_agent(observation, configuration):
    """Агент-предсказатель, который выбирает ход на основе вероятности действий противника.

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: число из диапазона [0, configuration.signs - 1].
    """
    if observation.step == 0:
        return random.randrange(0, configuration.signs)  
        
    opponent_move = observation.lastOpponentAction
    best_move = (opponent_move + 1) % configuration.signs  

    if random.random() < 0.8:  
        move = best_move
    else:
        move = random.randrange(0, configuration.signs) 
    printl(f"Agent action {move}")
    return move

reactionary_agent_last_react_action = None
def reactionary_agent(observation, configuration):
    """Агент, который реагирует на ход противника, выбирая жест, который противодействует его предыдущему ходу.

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: число из диапазона [0, configuration.signs - 1].
    """
    global reactionary_agent_last_react_action
    if observation.step == 0:
        reactionary_agent_last_react_action = random.randrange(0, configuration.signs)
    elif get_score(reactionary_agent_last_react_action, observation.lastOpponentAction) <= 1:
        reactionary_agent_last_react_action = (observation.lastOpponentAction + 1) % configuration.signs
    move = reactionary_agent_last_react_action
    printl(f"Agent action {move}")
    return move


counter_reactionary_agent_last_counter_action = None
def counter_reactionary_agent(observation, configuration):
    """Агент, который реагирует на ход противника, используя стратегию контр-реакции.

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: число из диапазона [0, configuration.signs - 1].
    """
    global counter_reactionary_agent_last_counter_action
    if observation.step == 0:
        counter_reactionary_agent_last_counter_action = random.randrange(0, configuration.signs)
    elif get_score(counter_reactionary_agent_last_counter_action, observation.lastOpponentAction) == 1:
        counter_reactionary_agent_last_counter_action = (counter_reactionary_agent_last_counter_action + 2) % configuration.signs
    else:
        counter_reactionary_agent_last_counter_action = (observation.lastOpponentAction + 1) % configuration.signs
    move = counter_reactionary_agent_last_counter_action
    printl(f"Agent action {move}")
    return move

statistical_agent_action_histogram = {}
def statistical_agent(observation, configuration):
    """Агент, который реагирует на ход противника, основываясь на статистике предыдущих ходов.

    Args:
        observation (object): Наблюдение о состоянии раунда:
            - observation.step (int): номер шага.
            - observation.reward (int): величина награды.
        configuration (object): Конфигурация игры:
            - configuration.signs (int): количество возможных жестов.

    Returns:
        int: число из диапазона [0, configuration.signs - 1].
    """
    global statistical_agent_action_histogram
    if observation.step == 0:
        statistical_agent_action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in statistical_agent_action_histogram:
        statistical_agent_action_histogram[action] = 0
    statistical_agent_action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in statistical_agent_action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue
    move = (mode_action + 1) % configuration.signs
    printl(f"Agent action {move}")
    return move

In [9]:
agents = [
    rock_agent,
    paper_agent,
    scissors_agent,
    random_agent,
    copy_opponent_agent,
    respond_to_opponent_agent,
    agent_sequence,
    common_selection_agent,
    reward_check_agent,
    anti_replay_agent,
    predictive_agent,
    reactionary_agent,
    counter_reactionary_agent,
    statistical_agent,
]
print(f"Всего агентов: {len(agents)}")
results = {agent.__name__: 0 for agent in agents}
match_results = []

for i in range(len(agents)):
    for j in range(len(agents)):
        if i == j:
            continue
        rewards = evaluate(
            "rps",
            [agents[i], agents[j]],
            configuration={"episodeSteps": 200},
            debug=False,
        )[0]

        match_results.append(
            (agents[i].__name__, agents[j].__name__, rewards[0], rewards[1])
        )

        if rewards[0] > rewards[1]:
            results[agents[i].__name__] += 1
        elif rewards[1] > rewards[0]:
            results[agents[j].__name__] += 1

for agent_name, score in sorted(
    results.items(), key=lambda item: item[1], reverse=True
):
    print(f"{agent_name}: {score} побед")

df_rewards = pd.DataFrame(
    match_results, columns=["Agent 1", "Agent 2", "Rewards Agent 1", "Rewards Agent 2"]
)
results_list = []

for agent in {agent.__name__: 0 for agent in agents}:
    rewards_agent1, rewards_agent2 = (
        df_rewards[df_rewards["Agent 1"] == agent]["Rewards Agent 1"],
        df_rewards[df_rewards["Agent 2"] == agent]["Rewards Agent 2"],
    )
    results_list.append(
        {
            "Agent": agent,
            "Average Reward": pd.concat([rewards_agent1, rewards_agent2]).mean(),
        }
    )
pd.DataFrame(results_list).sort_values(by="Average Reward", ascending=False)

Всего агентов: 14
counter_reactionary_agent: 15 побед
anti_replay_agent: 13 побед
reactionary_agent: 12 побед
respond_to_opponent_agent: 11 побед
reward_check_agent: 10 побед
predictive_agent: 10 побед
statistical_agent: 8 побед
copy_opponent_agent: 6 побед
common_selection_agent: 5 побед
scissors_agent: 3 побед
rock_agent: 2 побед
paper_agent: 2 побед
random_agent: 2 побед
agent_sequence: 2 побед


,Agent,Average Reward
12,counter_reactionary_agent,61.000000
11,reactionary_agent,52.653846
9,anti_replay_agent,38.307692
10,predictive_agent,35.384615
5,respond_to_opponent_agent,34.884615
13,statistical_agent,16.038462
6,agent_sequence,14.423077
4,copy_opponent_agent,3.423077
3,random_agent,-0.076923
8,reward_check_agent,-15.115385


## Вывод

Топ-3 агентов по количеству побед:

* counter_reactionary_agent: 15 побед
* anti_replay_agent: 13 побед
* reactionary_agent: 12 побед

Топ-3 агентов по среднему reward:

* counter_reactionary_agent	61.000000
* reactionary_agent	52.653846
* anti_replay_agent	38.307692

Тактика, используемая агентом `counter_reactionary_agent`, оказалась наиболее эффективной.